In [18]:
# !pip install neuralprophet --upgrade
# !pip install python-dotenv

In [15]:
from comet_ml import Experiment
from comet_ml.api import API, APIExperiment
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, save, load
import pickle
import os
from dotenv import load_dotenv


In [9]:


load_dotenv()

COMET_ML_API_KEY = os.getenv('COMET_ML_API_KEY')
experiment = Experiment(
    # api_key=os.environ.get('COMET_ML_API_KEY'),
    api_key=COMET_ML_API_KEY,
    project_name="daily-visits",
    workspace="drdevinhopkins",
)
api = API(os.environ.get('COMET_ML_API_KEY'))


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/drdevinhopkins/daily-visits/da40b4f108b649dea951fb9ccbddfad9



In [10]:

set_log_level("ERROR")

df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds')


In [11]:

params = {
    # growth='off',
    # 'yearly_seasonality':False,
    # 'weekly_seasonality':True,
    # 'daily_seasonality':True,
    'n_lags': 4,
    'n_forecasts': 12,
    'changepoints_range': 0.95,
    'n_changepoints': 50,
    'quantiles': [0.2, 0.5, 0.8]
    # num_hidden_layers=4,
    # d_hidden=36,
    # learning_rate=0.005,
}

m = NeuralProphet(**params)

m = m.add_country_holidays("CA")

metrics = m.fit(df,
                freq='D',
                # progress='plot'
                )
print(metrics.tail(1))


Finding best initial lr:   0%|          | 0/240 [00:00<?, ?it/s]

Missing logger folder: /Users/devinhopkins/Dropbox/Code/hourly-report-neuralprophet/notebooks/lightning_logs


Training: 0it [00:00, ?it/s]

           MAE      RMSE      Loss  RegLoss  epoch
130  14.061237  18.20447  0.005401      0.0    130


In [14]:

experiment.log_parameters(params)
experiment.log_metrics(metrics.tail(1).iloc[0].to_dict())

save(m, "../models/daily-visits.np")

experiment.log_model("daily-visits", "../models/daily-visits.np")

# with open('../models/daily_visits_forecast_model.pkl', "wb") as f:
#     pickle.dump(m, f)

experiment_name = experiment.name

experiment.end()


api.delete_registry_model('drdevinhopkins', 'daily-visits')

apiexp = api.get('drdevinhopkins/daily-visits/'+experiment_name)

apiexp.register_model('daily-visits')


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/drdevinhopkins/daily-visits/da40b4f108b649dea951fb9ccbddfad9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Loss [134]  : (0.005400646943598986, 2.078678846359253)
COMET INFO:     MAE [134]   : (14.061237335205078, 322.8252258300781)
COMET INFO:     RMSE [134]  : (18.197662353515625, 377.7602844238281)
COMET INFO:     RegLoss     : 0.0
COMET INFO:     epoch [134] : (0, 130)
COMET INFO:     hp_metric   : -1
COMET INFO:     loss [1622] : (0.0036984067410230637, 2.415658712387085)
COMET INFO:   Parameters:
COMET INFO:     changepoints_range : 0.95
COMET INFO:     n_changepoints     : 50
COMET INFO:     n_forecasts        : 12
COMET INFO:     n_lags             : 4
COMET INFO:     quantiles          : [0.2, 0.5, 0.8]
COMET INFO:   Uploads:

{'registryModelId': 'FPWnMFcKwALMsby6mQf0t021e',
 'registryModelItemId': 'Dq9VFmNYCx6ikNLwirlPHh7Zi'}

In [17]:
# loaded_model = pickle.load(
#     open('models/daily_visits_forecast_model.pkl', 'rb'))
loaded_model = load("../models/daily-visits.np")

df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds')
# df['ID'] = 'test'

# periods=m.n_forecasts, n_historic_predictions=False
future = loaded_model.make_future_dataframe(df, periods=8)

# forecast = loaded_model.predict(future, raw=True, decompose=False)

# start = forecast.values.tolist()[-1][0]
# forecast_length = len(forecast.values.tolist()[-1][1:])

# forecast_output = pd.DataFrame(columns=['ds', 'visits', 'timestamp'])
# forecast_output['ds'] = pd.date_range(
#     start=start, periods=forecast_length, freq='D')
# forecast_output['visits'] = forecast.values.tolist()[-1][1:]
# forecast_output['timestamp'] = pd.Timestamp.now().round(
#     'S').replace(tzinfo=None)
# forecast_output.to_csv('forecasts/daily_visits.csv', index=False)


forecast = loaded_model.predict(future,
                                # raw=False,
                                # decompose=False
                                )

forecast_trimmed = forecast[forecast.y.isnull()].set_index('ds')
forecast_trimmed.to_csv('../forecasts/daily_visits_raw.csv')

start = forecast_trimmed.index.tolist()[0]
forecast_length = len(forecast_trimmed.index.tolist())
i = 0
timestamp = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output_list = []
for ds in pd.date_range(start=start, periods=forecast_length, freq='D'):
    i = i+1
    forecast_output_list.append(
        {'ds': ds, 'visits': forecast_trimmed.loc[ds]['yhat'+str(i)], 'timestamp': timestamp})
forecast_output = pd.DataFrame(forecast_output_list)
forecast_output.to_csv('../forecasts/daily_visits.csv', index=False)

Predicting: 0it [00:00, ?it/s]